In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
! unzip '/content/drive/MyDrive/Dataset.zip'

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
import tensorflow.keras.backend as K
from skimage import io
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input, Flatten, Dense, MaxPooling2D, Dropout
from tensorflow.keras.applications import VGG19

from warnings import filterwarnings
filterwarnings('ignore')
from google.colab.patches import cv2_imshow
import random

import glob
from IPython.display import display
from PIL import Image

In [ ]:
def dataset(data_path):
  train_dict, test_dict = {'image_name':[],'label':[]}, {'image_name':[],'label':[]}
  for category in os.listdir(data_path):
      cat_pathes = glob.glob(data_path+'/'+category+'/Train/*).png')
      train_dict['image_name']+=cat_pathes
      train_dict['label']+=[category]*len(cat_pathes)


      cat_pathes = glob.glob(data_path+'/'+category+'/Test/*).png')
      test_dict['image_name']+=cat_pathes
      test_dict['label']+=[category]*len(cat_pathes)
  train_df, test_df = pd.DataFrame.from_dict(train_dict).sample(frac=1), pd.DataFrame.from_dict(test_dict).sample(frac=1)

  return train_df, test_df

In [ ]:
data_path = '/content/Dataset/Breast scans'
train_df, test_df = dataset(data_path)

In [ ]:
train_df['label'].value_counts(normalize=True)

benign       0.584435
malignant    0.264317
normal       0.151248
Name: label, dtype: float64

In [ ]:
test_df['label'].value_counts(normalize=True)

benign       0.393939
normal       0.303030
malignant    0.303030
Name: label, dtype: float64

In [ ]:
FAST_RUN = True
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
batch_size = 32
LR = 1e-5

In [ ]:
class_weights = compute_class_weight(class_weight='balanced',classes=train_df['label'].unique(), y=train_df['label'])
class_weight = dict(zip([0,1,2], class_weights))
class_weight[1]+=1

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255, preprocessing_function=tf.keras.applications.mobilenet.preprocess_input
                                   ,zoom_range=0.3,vertical_flip=True, horizontal_flip=True,
                                   width_shift_range=0.2, height_shift_range=0.2,fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "", 
    x_col='image_name',
    y_col='label',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

Found 681 validated image filenames belonging to 3 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0/255, preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)
                                  #  ,zoom_range=0.3,vertical_flip=True, horizontal_flip=True,
                                  #  width_shift_range=0.2, height_shift_range=0.2,fill_mode='nearest')

test_generator = test_datagen.flow_from_dataframe(
    test_df, 
    "", 
    x_col='image_name',
    y_col='label',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

Found 99 validated image filenames belonging to 3 classes.


v1 weights model

opt = tf.keras.optimizers.Adam(1e-4)#1e-5.5  lr=0.0000031623
input_shape = (IMAGE_WIDTH,IMAGE_HEIGHT,IMAGE_CHANNELS)
base_model = tf.keras.applications.mobilenet.MobileNet(include_top=False, weights='imagenet',
                                                    input_shape=input_shape)

base_model.trainable=False

X = Conv2D(filters = 2048, kernel_size = 1, padding='valid', name='conv_1')(base_model.output)
X = Flatten()(X)
X = Dense(1024,activation='relu', name='dense_1')(X)
X = Dense(512,activation='relu', name='dense_2')(X)
X = Dense(256,activation='relu', name='dense_3')(X)
X = Dense(128,activation='relu', name='dense_4')(X)
X = Dense(64,activation='relu', name='dense_5')(X)
output = Dense(3,activation='sigmoid')(X)
model = Model(inputs=base_model.input, outputs=output)

333333333333333333333333333333333333333333333333333333333333333333

In [ ]:
opt = tf.keras.optimizers.Adam(1e-4)#1e-5.5  lr=0.0000031623
input_shape = (IMAGE_WIDTH,IMAGE_HEIGHT,IMAGE_CHANNELS)
base_model = tf.keras.applications.mobilenet.MobileNet(include_top=False, weights=None,
                                                    input_shape=input_shape)

X = Conv2D(filters = 2048, kernel_size = 1, padding='valid', name='conv_1')(base_model.output)
X = Flatten()(X)
X = Dense(1024,activation='relu')(X)
X = Dense(512,activation='relu')(X)
X = Dense(256,activation='relu')(X)
X = Dense(128,activation='relu')(X)
X = Dense(64,activation='relu')(X)
output = Dense(3,activation='sigmoid')(X)
model = Model(inputs=base_model.input, outputs=output)

model.load_weights('/content/drive/MyDrive/breast_calssifier.h5')

model.trainable=False
for layer in model.layers[-3:]:
    layer.trainable=True

model.compile(loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC()],
              optimizer=opt)
# model.save()

In [ ]:
# opt = tf.keras.optimizers.Adam(lr=1e-3)
# input_shape = IMAGE_SIZE + (3,) 

# input_layer = tf.keras.Input(shape=input_shape)
# X = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(input_layer)
# X = Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu')(X)

# X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
# X = Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(X)
# X = Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu')(X)

# X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
# X = Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(X)
# X = Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(X)
# X = Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu')(X)
# X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)

# X = Flatten()(X)
# X = Dense(units=2048, activation='relu')(X)
# X = Dropout(0.6)(X)
# X = Dense(units=2048, activation='relu')(X)
# output = Dense(3,activation='sigmoid')(X)
# model = Model(inputs=input_layer, outputs=output)

# model.compile(loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC()],
#               optimizer=opt)

In [ ]:
# opt = tf.keras.optimizers.Adam(1e-4)#1e-5.5  lr=0.0000031623
# input_shape = (IMAGE_WIDTH,IMAGE_HEIGHT,IMAGE_CHANNELS)
# base_model = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet',
#                                                     input_shape=input_shape)

# base_model.trainable=False
# # for layer in base_model.layers[-5:]:
# #   layer.trainable=True

# X = Conv2D(filters = 2048, kernel_size = 1, padding='valid', name='conv_1')(base_model.output)
# X = Conv2D(filters = 1024, kernel_size = 1, padding='valid')(X)
# # X = Conv2D(filters = 512, kernel_size = 1, padding='valid')(X)
# X = Flatten()(X)
# X = Dense(1024,activation='relu', name='dense_1')(X)
# X = Dense(512,activation='relu', name='dense_2')(X)
# X = Dense(256,activation='relu', name='dense_3')(X)
# X = Dense(128,activation='relu', name='dense_4')(X)
# X = Dense(64,activation='relu', name='dense_5')(X)
# output = Dense(3,activation='sigmoid')(X)
# model = Model(inputs=base_model.input, outputs=output)

# # model.summary()

# model.compile(loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC()],
#               optimizer=opt)

In [ ]:
# layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
# pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable']).tail(50)

In [ ]:
checkpoint_filepath = '/content/drive/MyDrive/CV_project/breast_classifier_v5.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_auc',
    mode='max')
callbacks = [model_checkpoint_callback]

In [ ]:
total_train = train_df.shape[0]
total_validate = test_df.shape[0]
batch_size=32

In [ ]:
epochs=5 if FAST_RUN else 10
history = model.fit(train_generator, epochs=epochs,
    validation_data=test_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks,
    class_weight=class_weight
)

Epoch 1/5
21/21 [==============================] - 19s 870ms/step - loss: 0.3590 - auc_5: 0.9373 - val_loss: 0.8197 - val_auc_5: 0.8607
Epoch 2/5
21/21 [==============================] - 19s 925ms/step - loss: 0.2870 - auc_5: 0.9423 - val_loss: 0.8239 - val_auc_5: 0.8614
Epoch 3/5
21/21 [==============================] - 18s 897ms/step - loss: 0.3562 - auc_5: 0.9432 - val_loss: 0.8278 - val_auc_5: 0.8613
Epoch 4/5
21/21 [==============================] - 18s 896ms/step - loss: 0.2970 - auc_5: 0.9415 - val_loss: 0.8399 - val_auc_5: 0.8584
Epoch 5/5
21/21 [==============================] - 17s 790ms/step - loss: 0.3872 - auc_5: 0.9368 - val_loss: 0.7483 - val_auc_5: 0.8653


In [ ]:
model.evaluate(test_generator)

4/4 [==============================] - 1s 187ms/step - loss: 0.8167 - auc: 0.8624


[0.8167282342910767, 0.8624374866485596]